In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import subprocess
from ultralytics import YOLO
import time

In [2]:
# FFmpeg command
ffmpeg_cmd = [
    "ffmpeg",
    "-f", "avfoundation",  # Use AVFoundation input
    "-framerate", "25",    # Set frame rate
    "-video_size", "256x384",  # Set resolution
    "-pixel_format", "yuyv422",  # Set input pixel format
    "-i", "0",  # Camera index (change if needed)
    "-f", "rawvideo",  # Output as raw video
    "-pix_fmt", "bgr24",  # Convert to OpenCV-friendly format
    "-"  # Output to stdout
]

# Start FFmpeg process
process = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL, bufsize=10**8)

width, height = 256, 384 # Set resolution

frame_number = 0
infer_time = []
start_time = time.time()

while True:
    # Read one frame from FFmpeg output
    frame_size = width * height * 3  # BGR24 has 3 channels
    frame_bytes = process.stdout.read(frame_size)

    if not frame_bytes:
        break  # End of stream

    # Convert bytes to numpy array
    frame = np.frombuffer(frame_bytes, np.uint8).reshape((height, width, 3))

    imdata,thdata = np.array_split(frame, 2)

    imdata = cv2.resize(imdata, (512,384), interpolation = cv2.INTER_CUBIC)

    # Show frame
    cv2.imshow("Camera", imdata)

    frame_number += 1

    if cv2.waitKey(1) & 0xFF == ord("q"):
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        break

end_time = time.time()
fps = frame_number / (end_time - start_time)
print(f"Processed {frame_number} frames")
print(f"Average FPS: {fps:.2f}")
print(f"Average inference time: {np.mean(infer_time):.2f} ms")

# Cleanup
process.terminate()
cv2.waitKey(1)
cv2.destroyAllWindows()

Processed 79 frames
Average FPS: 19.53
Average inference time: nan ms


/Users/ayushshrivastava/miniconda3/envs/python3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ayushshrivastava/miniconda3/envs/python3.9/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
